In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np

In [8]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [9]:
# get main shelter that sit on top of parent shelters => Main > Parent > Child

driver.get("https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/accueil.aspx")
page = driver.find_element_by_class_name('divWCAD304_FamilyList')


parent_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
parent_urls = parent_urls[2:10]
urls = []
for parent in parent_urls:
    sleep(1)
    driver.get(parent)
    sleep(1)
    try:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "divWCAD304_FamilyList")))
        page = driver.find_element_by_class_name('divWCAD304_FamilyList')
        a = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
        for url in a:
            urls.append(url)
    except:
        next

        
urls
    


['https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284326-Volailles-et-Gibiers.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284327-Poissonnerie.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-285187-Traiteur-de-la-mer.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284351-Legumes.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284350-Jus-de-fruits-et-legumes-frais.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284349-Fruits.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284358-Pains-et-Patisseries.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284361-Pains-de-mie-et-Pains-grilles.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284362-Patisse

In [10]:
url = "https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx"
driver.get(url)
driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-2].text[:-2]

'Viandes Poissons'

In [11]:
product_pages = []

for url in urls:
    driver.get(url)

    shelters = driver.find_element_by_class_name('divWCRS315_FiltresNavigation').find_element_by_css_selector('ul')
    shelter_main = driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-2].text[:-2]
    shelter_parent = driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-1].text
    links = [elem.get_attribute("href") for elem in shelters.find_elements_by_tag_name('a')]


    for link in links:
        sleep(1)
        if links.index(link) == 0:
            next
        else:      
            driver.get(link)

            actions = ActionChains(driver)
            actions.move_to_element(driver.find_element_by_id("ulListeProduits").find_elements_by_css_selector('li')[-1]).perform()

            shelter_child = driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-1].text

            sleep(1)
            request_text = requests.get(link)
            soup = BeautifulSoup(request_text.text,"html.parser" ,from_encoding="utf-8")


            brands = []
            try:
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of_element_located((By.XPATH, "//dt[contains(text(),'Marques')]")))
                if driver.find_element_by_id('divBlocsFiltres').find_element_by_xpath("//dt[contains(text(),'Marques')]"):
                    for filter in driver.find_elements_by_class_name('dlCRS315_Filtre'):
                        if "Marques" in filter.find_element_by_css_selector("dt").text:
                            for label in filter.find_elements_by_css_selector('label'):
                                brands.append(label.text)
                product_pages.append({"url":link, 
                         "shelter_main": shelter_main,
                         "shelter_parent": shelter_parent,
                         "shelter_child": shelter_child,
                         "brands":brands,
                         "soup": soup, 
                         "text": driver.find_element_by_id("ulListeProduits").text
                        })
            except:
                product_pages.append({"url":link,
                     "shelter_main": shelter_main,
                     "shelter_parent": shelter_parent,
                     "shelter_child": shelter_child,
                     "brands":brands,
                     "soup": soup,
                     "text": driver.find_element_by_id("ulListeProduits").text
                    })


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:177: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [12]:
data = pd.DataFrame(product_pages)
data.to_csv('leclerc_catalog.csv')

In [18]:
import html
pages = []
for index, row in data.iterrows():
    soup = row.soup
    page = html.unescape(soup.text)
    page.find('objElement')
    page.find('lstAssocFiltreProduit')
    page = page[(page.find('objElement')-2):(page.find('lstAssocFiltreProduit')-2)]
    page = page.split("objElement")
    pages.append({"shelter_main": row.shelter_main,
                  "shelter_parent": row.shelter_parent,
                  "shelter_child": row.shelter_child,                  
                  "brands": row.brands,
                  "page": page[1:-1]
                })

In [19]:
products = []
for page in pages:
    for content in page['page']:
        item = content.split(",")
        for line in item:
            if "sLibelleLigne1" in line:
                description1 = line[18:-1]
            if "sLibelleLigne2" in line:
                description2 = line[18:-1]
            if "sPrixUnitaire" in line:
                price = line[17:-1]
            if "sPrixParUniteDeMesure" in line:
                price_per_unit = line[25:-1]
            if "sPrixPromo" in line:
                promo_price = line[14:-1]
            if "sUrlVignetteProduit" in line:
                image_url = line[23:-1]
            if "sOrigineZoneDepeche" in line:
                origin = line[23:-1]
            if "sUrlPageProduit" in line:
                url = line[19:-1]
            if "iIdProduit" in line:
                product_id = line[13:]
        description = description1 + " " + description2
        if len(page['brands']) > 0:
            brand = ''.join([brand for brand in page['brands'] if brand in description])
        else:
            brand = ""
                
        products.append({"shelter_main": page['shelter_main'],
                         "shelter_parent": page['shelter_parent'],
                         "shelter_child": page['shelter_child'],  
                         "description":description, 
                         "brand": brand,
                         "price": price,
                         "price_per_unit":price_per_unit,
                         "promo_price": promo_price,
                         "origin":origin,
                         "image_url": image_url,                    
                         "url": url,
                         "product_id": product_id
                        })

products

[{'shelter_main': 'Viandes Poissons',
  'shelter_parent': 'Boucherie',
  'shelter_child': 'Saucisses et Grillades',
  'description': 'Plateau grill Tendre et Plus Extra tendre - Herbes  x4 -650g',
  'brand': '',
  'price': '8.84 €',
  'price_per_unit': '13.60 € / kg',
  'promo_price': '8.84 €',
  'origin': '',
  'image_url': 'https://fd4-photos.leclercdrive.fr/image.ashx?id=1674766&use=l&cat=p&typeid=i',
  'url': 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/fiche-produits-104789-Plateau-grill-Tendre-et-Plus.aspx',
  'product_id': '104789'},
 {'shelter_main': 'Viandes Poissons',
  'shelter_parent': 'Boucherie',
  'shelter_child': 'Saucisses et Grillades',
  'description': 'Brochette porc Tendre Et Plus Extra tendre x4 - 320g',
  'brand': '',
  'price': '3.89 €',
  'price_per_unit': '12.16 € / kg',
  'promo_price': '3.89 €',
  'origin': '',
  'image_url': 'https://fd4-photos.leclercdrive.fr/image.ashx?id=1603250&use=l&cat=p&typeid=i',
  'url': 'https://fd4-courses.lecle

In [20]:
product_data = pd.DataFrame(products)

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_leclerc.csv', index = None, header=True)


In [21]:
print(len(product_data))
product_data

8173


,brand,description,image_url,origin,price,price_per_unit,product_id,promo_price,shelter_child,shelter_main,shelter_parent,url
0,,Plateau grill Tendre et Plus Extra tendre - He...,https://fd4-photos.leclercdrive.fr/image.ashx?...,,8.84 €,13.60 € / kg,104789,8.84 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
1,,Brochette porc Tendre Et Plus Extra tendre x4 ...,https://fd4-photos.leclercdrive.fr/image.ashx?...,,3.89 €,12.16 € / kg,48468,3.89 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
2,,Saucisses aux herbes Ferial Avec Chipolata,https://fd4-photos.leclercdrive.fr/image.ashx?...,,2.97 €,9.00 € / kg,53190,2.97 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
3,,Saucisse Chipolatas Bio x5 - VPF - 275g,https://fd4-photos.leclercdrive.fr/image.ashx?...,,4.07 €,14.80 € / kg,55810,4.07 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
4,Bigard,Chipolatas label rouge Bigard x6 - 330g,https://fd4-photos.leclercdrive.fr/image.ashx?...,,3.55 €,10.76 € / kg,20982,3.55 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
5,Férial,Chipolatas Férial 6x - 55g,https://fd4-photos.leclercdrive.fr/image.ashx?...,,2.99 €,9.06 € / kg,18218,2.99 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
6,Socopa,Chipolatas superieure Socopa Sous atmosphère x...,https://fd4-photos.leclercdrive.fr/image.ashx?...,,3.10 €,9.39 € / kg,32843,3.10 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
7,,Chipolatas Tendre Et Plus x6 - 330g,https://fd4-photos.leclercdrive.fr/image.ashx?...,,2.80 €,8.48 € / kg,46231,2.80 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
8,,Chipolatas supérieures x6 - 330g,https://fd4-photos.leclercdrive.fr/image.ashx?...,,3.69 €,11.18 € / kg,84763,3.69 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...
9,,Chipolatas boyau naturel - VPF x24 -,https://fd4-photos.leclercdrive.fr/image.ashx?...,,8.03 €,6.69 € / kg,84353,8.03 €,Saucisses et Grillades,Viandes Poissons,Boucherie,https://fd4-courses.leclercdrive.fr/magasin-12...


In [22]:
product_data.brand.unique()

array(['', 'Bigard', 'Férial', 'Socopa', 'Bio Village', 'Charal',
       'Tendre & Plus', 'Volandry', 'Le Gaulois', 'Loué', 'Père Dodu',
       'Ronsard', 'Montfort', 'Céréal', 'Herta', 'Ronde des mers',
       'Delpierre', 'Cap Océan', 'Labeyrie', 'Delpeyrat', 'Coraya',
       'Fleury Michon', 'Pêche Océan', 'Gastromer', 'Blini',
       'Notre Jardin', 'Bonduelle', 'Jafaden', 'Innocent', 'Andros',
       'Tropicana', 'Sunny Delight', 'Danao', 'Couleurs Vives',
       'Tablier Blanc', 'Céréal Bio', 'Bjorg', 'Gerblé', "Epi d'Or",
       'Lenôtre', 'Autour du Dessert', 'Trofic', 'Harrys', 'Jacquet',
       'Allergo', 'Pelletier', 'La Boulangère', 'Pasquier',
       'Nos régions ont du talent', 'Pitch', 'PasquierPitch', 'Whaou !',
       "P'tit déli", 'Jardin Bio', 'Brossard', 'Matines', 'Lustucru',
       'Lactel', 'Délisse', 'Cacolac', 'Nat & Vie', 'Grand Fermage',
       'Elle & Vire', 'Bridel', 'BridelBridelight', 'Le Gall',
       "Fruit d'Or", 'Le Fleurier', 'Bridelight', 'Nadya', '